In [1]:
import os
import re
import pandas as pd
import numpy as np
import qiime2
from qiime2 import Artifact
from qiime2.plugins import metadata, feature_table, diversity, emperor
from q2_emperor import plot, procrustes_plot, biplot, generic_plot
from scipy.spatial.distance import squareform, pdist
import skbio

In [2]:
os.chdir('..')

In [3]:
os.getcwd()

'/ClusterApp'

In [4]:
from api.src.service.gnps import Proteosafe

In [19]:
taskid = 'a0b4377f6e4540268602c30fedf69b0b'
workflow = 'FBMN'
metric = 'euclidean'
gnps_result = Proteosafe(taskid, workflow)
gnps_result.get_gnps()

In [20]:
gnps_result.meta

,filename,ATTRIBUTE_Sample_Number,ATTRIBUTE_Order_Injection_2965_3788,ATTRIBUTE_Plot,ATTRIBUTE_Forest_Region,ATTRIBUTE_Pertubation,ATTRIBUTE_Super_Order_APGIV_Clade,ATTRIBUTE_Order_APGIV_Order,ATTRIBUTE_Family,ATTRIBUTE_Genus,...,ATTRIBUTE_Order_APGIII,ATTRIBUTE_CS,ATTRIBUTE_AB,ATTRIBUTE_LIG,ATTRIBUTE_AFE,ATTRIBUTE_CMSF,ATTRIBUTE_Asat,ATTRIBUTE_C,ATTRIBUTE_N,ATTRIBUTE_C/N
0,10_3uL_30_05_1-33_01_3004.mzXML,10,3004,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
1,11_3uL_01_06_1-42_01_3131.mzXML,11,3131,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
2,12_3uL_30_05_1-68_01_3052.mzXML,12,3052,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,0.105,0.135,0.016,0.387,0.120,456.674,32.262,14.155,NaN,NaN
3,78_3uL_03_07_1-66_01_3757.mzXML,78,3757,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.116,0.211,0.010,0.347,0.064,484.183,23.933,20.231,NaN,NaN
4,79_3uL_03_07_1-17_01_3699.mzXML,79,3699,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.116,0.211,0.010,0.347,0.064,484.183,23.933,20.231,NaN,NaN
5,80_3uL_30_05_1-58_01_3033.mzXML,80,3033,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.116,0.211,0.010,0.347,0.064,484.183,23.933,20.231,NaN,NaN
6,152_3uL_01_06_1-87_01_3074.mzXML,152,3074,N,Secondary,Selective,Monocots,Arecales,Arecaceae,Euterpe,...,0.094,0.135,0.007,0.288,0.030,464.898,20.902,22.241,NaN,NaN
7,153_3uL_01_06_1-23_01_3111.mzXML,153,3111,N,Secondary,Selective,Monocots,Arecales,Arecaceae,Euterpe,...,0.094,0.135,0.007,0.288,0.030,464.898,20.902,22.241,NaN,NaN
8,154_3uL_01_06_1-37_01_3125.mzXML,154,3125,N,Secondary,Selective,Monocots,Arecales,Arecaceae,Euterpe,...,0.094,0.135,0.007,0.288,0.030,464.898,20.902,22.241,NaN,NaN
9,224_3uL_03_07_1-81_01_3774.mzXML,224,3774,M,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,0.074,0.132,0.009,0.358,0.048,459.564,24.941,18.426,NaN,NaN


In [21]:
feat = gnps_result.feat
feat.head()

,row ID,row m/z,row retention time,12_3uL_30_05_1-68_01_3052.mzXML Peak area,78_3uL_03_07_1-66_01_3757.mzXML Peak area,11_3uL_01_06_1-42_01_3131.mzXML Peak area,10_3uL_30_05_1-33_01_3004.mzXML Peak area,80_3uL_30_05_1-58_01_3033.mzXML Peak area,153_3uL_01_06_1-23_01_3111.mzXML Peak area,152_3uL_01_06_1-87_01_3074.mzXML Peak area,79_3uL_03_07_1-17_01_3699.mzXML Peak area,224_3uL_03_07_1-81_01_3774.mzXML Peak area,154_3uL_01_06_1-37_01_3125.mzXML Peak area,225_3uL_03_07_1-92_01_3787.mzXML Peak area,226_3uL_03_07_1-6_01_3686.mzXML Peak area,Unnamed: 15
0,5,353.265401,35.119569,1.099282e+06,1631616.110,1.281668e+06,860049.175,1254956.950,1.010253e+06,900838.985,1.441528e+06,1.391126e+06,1.317142e+06,1.292446e+06,2443.4200,NaN
1,7,381.296298,37.262528,9.998174e+05,1445426.520,1.249140e+06,915588.990,1234167.890,9.921647e+05,800190.520,1.270731e+06,1.384528e+06,1.256917e+06,1.198393e+06,303.0000,NaN
2,8,581.148154,7.884651,1.348523e+06,1536205.639,5.228738e+06,1779064.237,127515.698,1.108239e+05,36034.871,2.423437e+04,1.746068e+06,1.812014e+05,3.515917e+04,338.3965,NaN
3,10,609.268397,35.247439,5.505347e+05,1259964.695,1.099087e+06,1900706.545,525107.070,4.631662e+05,325802.580,3.167505e+05,3.375690e+06,6.051916e+05,1.972567e+06,0.0000,NaN
4,13,305.156498,49.782424,6.643086e+05,299720.325,8.127709e+05,559341.440,714114.810,9.085950e+05,683273.360,4.172964e+05,3.073419e+05,7.009724e+05,4.861288e+05,0.0000,NaN


In [22]:
merged_df = pd.concat([gnps_result.meta, gnps_result.feat], axis=1)

In [23]:
merged_df.head()

,filename,ATTRIBUTE_Sample_Number,ATTRIBUTE_Order_Injection_2965_3788,ATTRIBUTE_Plot,ATTRIBUTE_Forest_Region,ATTRIBUTE_Pertubation,ATTRIBUTE_Super_Order_APGIV_Clade,ATTRIBUTE_Order_APGIV_Order,ATTRIBUTE_Family,ATTRIBUTE_Genus,...,10_3uL_30_05_1-33_01_3004.mzXML Peak area,80_3uL_30_05_1-58_01_3033.mzXML Peak area,153_3uL_01_06_1-23_01_3111.mzXML Peak area,152_3uL_01_06_1-87_01_3074.mzXML Peak area,79_3uL_03_07_1-17_01_3699.mzXML Peak area,224_3uL_03_07_1-81_01_3774.mzXML Peak area,154_3uL_01_06_1-37_01_3125.mzXML Peak area,225_3uL_03_07_1-92_01_3787.mzXML Peak area,226_3uL_03_07_1-6_01_3686.mzXML Peak area,Unnamed: 15
0,10_3uL_30_05_1-33_01_3004.mzXML,10.0,3004.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,860049.175,1254956.950,1.010253e+06,900838.985,1.441528e+06,1.391126e+06,1.317142e+06,1.292446e+06,2443.4200,NaN
1,11_3uL_01_06_1-42_01_3131.mzXML,11.0,3131.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,915588.990,1234167.890,9.921647e+05,800190.520,1.270731e+06,1.384528e+06,1.256917e+06,1.198393e+06,303.0000,NaN
2,12_3uL_30_05_1-68_01_3052.mzXML,12.0,3052.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,1779064.237,127515.698,1.108239e+05,36034.871,2.423437e+04,1.746068e+06,1.812014e+05,3.515917e+04,338.3965,NaN
3,78_3uL_03_07_1-66_01_3757.mzXML,78.0,3757.0,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,1900706.545,525107.070,4.631662e+05,325802.580,3.167505e+05,3.375690e+06,6.051916e+05,1.972567e+06,0.0000,NaN
4,79_3uL_03_07_1-17_01_3699.mzXML,79.0,3699.0,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,559341.440,714114.810,9.085950e+05,683273.360,4.172964e+05,3.073419e+05,7.009724e+05,4.861288e+05,0.0000,NaN


In [24]:
def _rename_columns(df):
        new_columns = {}
        for col in df.columns:
            if col.strip() == '' or col.startswith('Unnamed:'):
                df.drop(columns=[col], inplace=True)
            elif col not in ['row ID', 'row m/z', 'row retention time']:
                matches = re.findall(r'(\d+)', col)
                if len(matches) >= 2:
                    mz = matches[0]
                    rt = matches[1]
                    new_col_name = f"{mz}_{rt}"
                    new_columns[col] = new_col_name
                else:
                    raise ValueError(f"Column name '{col}' does not contain enough numeric parts to form 'mz_rt' format.")
        
        df.rename(columns=new_columns, inplace=True)
        
        df.drop(columns=['row ID', 'row m/z', 'row retention time'], inplace=True)
        
        return df

In [25]:
df = _rename_columns(gnps_result.feat)
df.head()

,12_3,78_3,11_3,10_3,80_3,153_3,152_3,79_3,224_3,154_3,225_3,226_3
0,1.099282e+06,1631616.110,1.281668e+06,860049.175,1254956.950,1.010253e+06,900838.985,1.441528e+06,1.391126e+06,1.317142e+06,1.292446e+06,2443.4200
1,9.998174e+05,1445426.520,1.249140e+06,915588.990,1234167.890,9.921647e+05,800190.520,1.270731e+06,1.384528e+06,1.256917e+06,1.198393e+06,303.0000
2,1.348523e+06,1536205.639,5.228738e+06,1779064.237,127515.698,1.108239e+05,36034.871,2.423437e+04,1.746068e+06,1.812014e+05,3.515917e+04,338.3965
3,5.505347e+05,1259964.695,1.099087e+06,1900706.545,525107.070,4.631662e+05,325802.580,3.167505e+05,3.375690e+06,6.051916e+05,1.972567e+06,0.0000
4,6.643086e+05,299720.325,8.127709e+05,559341.440,714114.810,9.085950e+05,683273.360,4.172964e+05,3.073419e+05,7.009724e+05,4.861288e+05,0.0000


In [27]:
merged_df = pd.concat([gnps_result.meta, df], axis=1)
merged_df.head()

,filename,ATTRIBUTE_Sample_Number,ATTRIBUTE_Order_Injection_2965_3788,ATTRIBUTE_Plot,ATTRIBUTE_Forest_Region,ATTRIBUTE_Pertubation,ATTRIBUTE_Super_Order_APGIV_Clade,ATTRIBUTE_Order_APGIV_Order,ATTRIBUTE_Family,ATTRIBUTE_Genus,...,11_3,10_3,80_3,153_3,152_3,79_3,224_3,154_3,225_3,226_3
0,10_3uL_30_05_1-33_01_3004.mzXML,10.0,3004.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,1.281668e+06,860049.175,1254956.950,1.010253e+06,900838.985,1.441528e+06,1.391126e+06,1.317142e+06,1.292446e+06,2443.4200
1,11_3uL_01_06_1-42_01_3131.mzXML,11.0,3131.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,1.249140e+06,915588.990,1234167.890,9.921647e+05,800190.520,1.270731e+06,1.384528e+06,1.256917e+06,1.198393e+06,303.0000
2,12_3uL_30_05_1-68_01_3052.mzXML,12.0,3052.0,T,Secondary,Clearcut,Monocots,Arecales,Arecaceae,Euterpe,...,5.228738e+06,1779064.237,127515.698,1.108239e+05,36034.871,2.423437e+04,1.746068e+06,1.812014e+05,3.515917e+04,338.3965
3,78_3uL_03_07_1-66_01_3757.mzXML,78.0,3757.0,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,1.099087e+06,1900706.545,525107.070,4.631662e+05,325802.580,3.167505e+05,3.375690e+06,6.051916e+05,1.972567e+06,0.0000
4,79_3uL_03_07_1-17_01_3699.mzXML,79.0,3699.0,K,Primary,Preserved,Monocots,Arecales,Arecaceae,Euterpe,...,8.127709e+05,559341.440,714114.810,9.085950e+05,683273.360,4.172964e+05,3.073419e+05,7.009724e+05,4.861288e+05,0.0000


In [10]:
merged_df.to_csv('file.csv')